# Social Media Monitoring

## Politician List


The following table contains data on the politicians and their associated social acounts

In [ ]:
import qgrid
from smm_wrapper import SMMPoliticians
qgrid.set_grid_option('maxVisibleRows', 5)

# Create an instance to the smm wrapper
smm = SMMPoliticians()

# Request the politicians from the API
df = smm.dv.get_politicians()

# Show the result
qgrid.show_grid(df)

## Instructions

Please choose an `affiliation` party from the table above and enter its name in the field below:

In [ ]:
party = 'AfD'
aggregation = 'month' #aggregation can be by day ('day'), week ('week') or month('month')

In [ ]:
import pandas as pd

# Filter the accounts by party, and valid ones (the ones that contain fb_ids)
party_df = df[(df['affiliation']==party) & (df['fb_ids'].notnull())] 

# query the Social Media Monitoring API
posts_by = pd.concat(smm.dv.posts_by(facebook_user_id=fb_id, from_date='2017-09-01', to_date='2018-12-31', aggregate_by=aggregation) 
               for fb_ids in party_df['fb_ids'] for fb_id in fb_ids)        
comments_by = pd.concat(smm.dv.comments_by(facebook_user_id=fb_id, from_date='2017-09-01', to_date='2018-12-31', aggregate_by=aggregation) 
               for fb_ids in party_df['fb_ids'] for fb_id in fb_ids) 

# aggregate the posts
total_posts_by = posts_by.groupby('date').agg(
    {
        'posts': sum,
        'replies': sum, 
        'shares': sum,
        'reactions': sum,
        'likes': sum
    })
# aggregate the comments
total_comments_by = comments_by.groupby('date').agg(
    {
        'comments': sum,
        'replies': sum,
        'likes': sum
    })

## Plotting

In [ ]:
import plotly
from plotly import graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

#plot for fb posts
plotly.offline.iplot({
    "data": [go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['posts'], name='Posts'), 
             go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['replies'], name='Replies'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['shares'], name='Shares'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['reactions'], name='Reactions'),
            go.Scatter(x=total_posts_by.index.tolist(), y=total_posts_by['likes'], name='Likes')], 
    "layout": go.Layout(title='Facebook posts for {} party'.format(party), 
                        xaxis={'title':''}, yaxis={'title':'Posts'})
})
#plot for fb comments
plotly.offline.iplot({
    "data": [go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['comments'], name='Comments'), 
             go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['replies'], name='Replies'),
            go.Scatter(x=total_comments_by.index.tolist(), y=total_comments_by['likes'], name='Likes')], 
    "layout": go.Layout(title='Facebook comments for {} party'.format(party), 
                        xaxis={ 'title':''}, yaxis={'title':'Comments'})
})